In [1]:

import os
import os.path as osp
import json
import pickle
import collections as C
import itertools as I

from common.pantograph.dataclasses import ProblemGenerationProcess

In [2]:
split = 'PromptCoT-DS'
# split = 'PromptCoT-QwQ'
# split = 'ScaleQuest-Math'


In [3]:
with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/autoformalization_pg_kimina7b/{split}.pkl', 'rb') as f:
    data = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/autoformalization_pg_kimina7b/PromptCoT-DS.pkl'

In [45]:
data_informal = [
    dict(
        informal_problem=d.informal_problem,
        informal_answer=d.informal_answer,
        informal_solution=d.informal_solution,
        header=d.header,
        formal_statement=d.formal_statement,
        metainfo=d.metainfo
    ) for d in data
]

In [46]:
C.Counter(
    len(d.formal_statement or '') == 0 for d in data
)

Counter({False: 3238, True: 1762})

In [ ]:
# with open(osp.join('/cache/data/fpg_informal_baselines/sampled', f'{split}.jsonl'), 'w') as f:
#     for d in data_informal:
#         f.write(json.dumps(d)+'\n')

### MUSTARDSauce_lean4_parsed

In [5]:
with open('/cache/data/MUSTARDSauce_lean4_parsed/20250910-155831.pkl', 'rb') as f:
    data = pickle.load(f)
print(len(data))

5858


In [6]:
with open('/cache/data/fpg_informal_baselines/sampled/MUSTARDSauce.processed.jsonl', 'r') as f:
    data_informal = [json.loads(l) for l in f.readlines()]
data_informal = [d for d in data_informal if d['lean4_source_path'] is not None]
print(len(data_informal))

5858


In [7]:
assert len(data) == len(data_informal)
for d, di in zip(data, data_informal):
    for k in set(d.keys()).intersection(di.keys()):
        assert d[k] == di[k]
    d |= di

In [8]:
d

{'informal_statement': 'Maria has a goal to read 100 books this year. She has already read 35 books. If Maria reads 5 books every week, how many more weeks does she need to reach her goal?',
 'formal_proof': 'open nat\ndef books_to_read : ℕ := 100 - 35\ndef weeks_needed : ℕ := books_to_read / 5\n#eval weeks_needed',
 'source_path': '/cache/data/MUSTARDSauce/data/subset_filtered',
 'source_idx': 'correct_by_msg__ELEM_word_problem_2nd_grade_877_round1',
 'file_name': 'wp_ELEM_k2_2nd_877_correct.json',
 'lean4_source_path': 'WpELEMK22nd877Correct.lean',
 'code_with_sorry': '',
 'parse_results': [],
 'informal_proof': 'First, we subtract the number of books Maria has already read from her goal: 100 - 35 = 65 books still need to be read. \n\nThen, we divide the remaining books by the number of books she reads every week: 65 ÷ 5 = 13 weeks. \n\nSo, Maria needs 13 more weeks to reach her goal.',
 'informal_answer': '13'}

In [9]:
all_parse_results = []
for d in data:
    for (i, u) in enumerate(d.get('parse_results', [])):
        assert u['code'].startswith('import Mathlib\nimport Aesop\n')
        formal_code = u['code'][len('import Mathlib\nimport Aesop\n'):].strip()
        all_parse_results.append(
            ProblemGenerationProcess(
                informal_problem=d['informal_statement'],
                informal_answer=d.get('informal_answer', ''),
                informal_solution=d.get('informal_proof', ''),
                header=None,
                formal_statement=formal_code,
                formal_solution_draft=u['proof'],
                formal_proofs=[],
                steps=[],
                dependencies=[],
                trajectory=[],
                metainfo={k : v for (k, v) in d.items() if k not in ['informal_statement', 'informal_answer', 'informal_proof', 'parse_results']} | {
                    'unit_idx': i
                }
            )
        )
len(all_parse_results)

3815

In [15]:
print(all_parse_results[33].formal_statement)
print('--------')
print(all_parse_results[33].formal_solution_draft)

-- Definition of the rectangle
-- Definition of the rectangle
structure Rectangle where
  length : ℝ
  width : ℝ
  l_pos : 0 < length
  w_pos : 0 < width

-- Perimeter of a rectangle
def perimeter (r : Rectangle) : ℝ :=
  2 * (r.length + r.width)

-- John's playground

def playground : Rectangle  := by
  sorry
--------

  ⟨50, 70, by linarith, by linarith⟩

-- Calculation of the needed fencing



In [16]:
print(all_parse_results[71].formal_statement)
print('--------')
print(all_parse_results[71].formal_solution_draft)

def calculatePerimeter (length : ℕ) (width : ℕ) : ℕ :=
  2 * (length + width)


def test_calculatePerimeter : calculatePerimeter 10 7 = 34  := by
  sorry
--------

  unfold calculatePerimeter
  norm_num



In [17]:
print(all_parse_results[80].formal_statement)
print('--------')
print(all_parse_results[80].formal_solution_draft)

theorem add_sub_same_inequality {a b c : ℝ} (h : a < b) : a - c < b - c  := by
  sorry
--------

  apply sub_lt_sub_right
  exact h



In [8]:
print(u['code'])

import Mathlib
import Aesop


-- Define the fractions
-- Define the fractions
noncomputable def johnsPiece : ℝ :=
  3 / 4

noncomputable def marysPiece : ℝ :=
  2 / 3

-- Prove that John's piece is larger

example : johnsPiece > marysPiece  := by
  sorry


In [9]:
print(u['proof'])


  unfold johnsPiece marysPiece
  -- Replace the definitions
  norm_num

-- Simplify the fractions


In [84]:
C.Counter(
    d.informal_problem == '' for d in all_parse_results
), C.Counter(
    d.informal_answer == '' for d in all_parse_results
), C.Counter(
    d.informal_solution == '' for d in all_parse_results
)

(Counter({False: 3815}),
 Counter({False: 2961, True: 854}),
 Counter({False: 2961, True: 854}))

In [85]:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/MUSTARDSauce_lean4_parsed/MUSTARDSauce_lean4.pkl', 'wb') as f:
    pickle.dump(all_parse_results, f)

In [48]:
all_to_prove = []
for d in data:
    all_to_prove.extend(d.get('parse_results', []))
len(all_to_prove)

3815

In [59]:
print(all_to_prove[-3000]['code'])

import Mathlib
import Aesop
-- import the necessary libraries


-- declare the variables
-- declare the variables
variable (x : ℝ)

-- declare the functions
def f (x : ℝ) :=
  2 * x ^ 2 + 5 * x - 3

def g (x : ℝ) :=
  x ^ 2 + 2 * x - 1

def h (x : ℝ) :=
  f x * g x

-- show the proof

example : h x = 2 * x ^ 4 + 9 * x ^ 3 + 5 * x ^ 2 - 11 * x + 3  := by
  sorry
